Connect to Weaviate

In [48]:
import weaviate

client = weaviate.connect_to_local()

client.is_ready()

True

Creating a collection in Weaviate that supports both Image and Text data <br>
Has 3 fields: <br>
"name", "image", "text" <br>
10%, 70%, 20% <br>
"name": name of image file <br>
"image": base64 encoding of image itself <br>
"text": any textual data or image metadata <br>

In [73]:
# Chunking
# Insert embedded text into weaviate

import weaviate.classes.config as wc

collection_name = "DemoCollection"  # Replace collection name here
collection_name_2 = "DemoCollection2"

# Check if the collection already exists and delete it if it does
if client.collections.exists(collection_name):
    client.collections.delete(collection_name)

# Check if the collection already exists and delete it if it does
if client.collections.exists(collection_name_2):
    client.collections.delete(collection_name_2)

client.collections.create(
        name=collection_name,
        properties=[
            wc.Property(name="name", data_type=wc.DataType.TEXT),
            wc.Property(name="image", data_type=wc.DataType.BLOB),
            wc.Property(name="text", data_type=wc.DataType.TEXT),
        ],
        # Define & configure the vectorizer module
        vectorizer_config=wc.Configure.Vectorizer.multi2vec_clip(
            image_fields=[wc.Multi2VecqField(name="image", weight=0.8)],    # 70% of the vector is from the image
            text_fields=[wc.Multi2VecField(name="name", weight=0.1),       # 10% of the vector is from the name
                         wc.Multi2VecField(name="text", weight=0.1)],      # 20% of the vector is from the text
        ),
    )

client.collections.create(
        name=collection_name_2,
        properties=[
            wc.Property(name="name2", data_type=wc.DataType.TEXT),
            wc.Property(name="image2", data_type=wc.DataType.BLOB),
            wc.Property(name="text2", data_type=wc.DataType.TEXT),
        ],
        # Define & configure the vectorizer module
        vectorizer_config=wc.Configure.Vectorizer.multi2vec_clip(
            image_fields=[wc.Multi2VecField(name="image2", weight=0.1)],    # 70% of the vector is from the image
            text_fields=[wc.Multi2VecField(name="name2", weight=0.1),       # 10% of the vector is from the name
                         wc.Multi2VecField(name="text2", weight=0.8)],      # 20% of the vector is from the text
        ),
    )

print(collection_name + " created")
print(collection_name_2 + " created")


DemoCollection created
DemoCollection2 created


In [50]:
import base64
import os
import requests

# Helper function to convert file to base64 representation
def to_base64(url_or_path):
    if url_or_path.startswith('http://') or url_or_path.startswith('https://'):
        # Handle URL
        image_response = requests.get(url_or_path)
        content = image_response.content
    elif os.path.exists(url_or_path):
        # Handle local file path
        with open(url_or_path, 'rb') as image_file:
            content = image_file.read()
    else:
        raise ValueError("The provided string is neither a valid URL nor a local file path.")
    
    return base64.b64encode(content).decode("utf-8")


Downloading images using simple image search

In [4]:
import argparse
from simple_image_download import simple_image_download as simp
from PIL import Image
import os

def image_download(query, number=5):
    # Initialize the argument parser
    parser = argparse.ArgumentParser(description="Download images.")
    parser.add_argument('query', type=str, help='Search query for images')
    parser.add_argument('number', type=int, nargs='?', default=5, help='Number of images to download (default: 5)')
    args = parser.parse_args([query, str(number)])

    # Extract arguments
    search_query = args.query
    num_images = args.number

    # Initialize the simple image download instance
    response = simp.simple_image_download

    # Specify the directory where images will be downloaded
    download_directory = 'downloads'

    # Download images for the specified query
    response().download(search_query, num_images)  # Download images for the search query

# Example usage
image_download("cat", 10)


[========================================================================] 100%


Insert images into Weaviate

In [75]:
collection = client.collections.get(collection_name)
collection_2 = client.collections.get(collection_name_2)

directory = "./simple_images/test_data/"
source = os.listdir(directory)
contextual_info = ["line graph, seasonal rainfall, pre-monsoon, monsoon, post-monsoon, 1971-2001",
                   "bar graph, mobile accesories orders analysis, pouches, holsters, shells, skins, fitted cases, bumpers, flip cases, sleeves",
                   "yearly expenses, insurance, utilities, groceries",
                   "regional review north america, wesley foods international, underlying revenue, organic revenue growth, underlying operating profit, underlying operating margin",
                   "ethnicity of students, asian, white, hispanic, other, black, filipino, pacific islander, native american",
                   ]

with collection.batch.dynamic() as batch:
    for x in range(len(source)):
    # for src_obj in source:
        src_obj = source[x]
        path = directory + src_obj
        poster_b64 = to_base64(path)
        weaviate_obj = {
            # "name": src_obj,
            "name": os.path.splitext(src_obj)[0],
            "image": poster_b64,  # Add the image in base64 encoding
            "text": contextual_info[x] # Optional text field for image metadata
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            # vector=vector  # Optionally provide a pre-obtained vector
        )

with collection_2.batch.dynamic() as batch:
    # for src_obj in source:
    for x in range(len(source)):
        src_obj = source[x]
        path = directory + src_obj
        poster_b64 = to_base64(path)
        weaviate_obj = {
            "name2": os.path.splitext(src_obj)[0],
            "image2": poster_b64,  # Add the image in base64 encoding
            "text2": contextual_info[x] # Optional text field for image data
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            # vector=vector  # Optionally provide a pre-obtained vector
        )
print("images added")

images added


Insert text into Weaviate

In [21]:
collection = client.collections.get(collection_name)

text_list = ["Cats in fact do not like dogs because they are enemies",
             "Eggplant is a natural enemy of the cat",
             "Cats love fish",
             "Monsters are enemies of battle cats",  
             "cats consider other cats to be their friends",
             "cats like hamsters",      
             ]

with collection.batch.dynamic() as batch:
    for text in text_list:
        weaviate_obj = {
            "text": text
        }

        # The model provider integration will automatically vectorize the object
        batch.add_object(
            properties=weaviate_obj,
            # vector=vector  # Optionally provide a pre-obtained vector
        )

Show objects in collection using Weaviate API

In [76]:
import requests
import json

# Define the Weaviate URL with query parameters
class_name = collection_name
limit = 10
url = f"http://localhost:8080/v1/objects?class={class_name}&limit={limit}"

# Send a GET request to fetch the objects
response = requests.get(url)

# Check if the request was successful
if response.status_code == 200:
    # Parse the JSON response
    objects = response.json()
    
    # Pretty-print the JSON response
    print(json.dumps(objects, indent=4))
else:
    # Print an error message if the request was not successful
    print(f"Failed to retrieve objects. Status code: {response.status_code}")
    print(response.text)

{
    "deprecations": [],
    "objects": [
        {
            "class": "DemoCollection",
            "creationTimeUnix": 1720425544168,
            "id": "46d52105-8f7f-44ba-b517-5778043c9c41",
            "lastUpdateTimeUnix": 1720425544168,
            "properties": {
                "image": "...",
                "name": "Ethnicity of students",
                "text": "ethnicity of students, asian, white, hispanic, other, black, filipino, pacific islander, native american"
            },
            "vectorWeights": null
        },
        {
            "class": "DemoCollection",
            "creationTimeUnix": 1720425544168,
            "id": "4be11d39-ac16-4ae8-9ca0-786f64d2594a",
            "lastUpdateTimeUnix": 1720425544168,
            "properties": {
                "image": "...",
                "name": "yearly expenses",
                "text": "yearly expenses, insurance, utilities, groceries"
            },
            "vectorWeights": null
        },
        {
   

Iterate through collection and print items

In [77]:
collection = client.collections.get(collection_name)

for item in collection.iterator(
    include_vector=True  # If using named vectors, you can specify ones to include e.g. ['title', 'body'], or True to include all
):
    print(item.properties)
    print(item.vector)

print("------")
collection_2 = client.collections.get(collection_name_2)

for item in collection_2.iterator(
    include_vector=True  # If using named vectors, you can specify ones to include e.g. ['title', 'body'], or True to include all
):
    print(item.properties)
    print(item.vector)


{'text': 'ethnicity of students, asian, white, hispanic, other, black, filipino, pacific islander, native american', 'name': 'Ethnicity of students'}
{'default': [0.052407752722501755, -0.032138410955667496, 0.004138434771448374, 0.12867142260074615, -0.0585712231695652, -0.1683744192123413, 0.07207275927066803, -0.045921165496110916, 0.2622693181037903, 0.00984009075909853, -0.013286237604916096, 0.08551222085952759, -0.1957685798406601, -0.023426277562975883, 0.10245388001203537, -0.015895431861281395, -0.04642890393733978, 0.04555496573448181, 0.02793901599943638, 0.1878150850534439, 0.14360815286636353, -0.03069985844194889, 0.08958835154771805, -0.04670817777514458, -0.08534226566553116, 0.16574853658676147, -0.11597016453742981, -0.03691089525818825, 0.02612333558499813, 0.10463381558656693, 0.03499201685190201, 0.03889238089323044, 0.061106178909540176, -0.0003416405525058508, 0.16456378996372223, 0.11722880601882935, 0.01990152709186077, 0.04258459806442261, -0.1062699556350708

Sample nearText query

In [83]:
import weaviate.classes as wvc
from weaviate.classes.query import Move

collection = client.collections.get(collection_name)

response = collection.query.near_text(
    query="bar graph",
    distance=0.6,
    return_metadata=wvc.query.MetadataQuery(distance=True),
    limit=2
)
# print(response)
# print(response.objects)

for o in response.objects:
    print(o.properties)
    print(o.metadata)

print("----------")
collection_2 = client.collections.get(collection_name_2)

response = collection_2.query.near_text(
    query="bar graph",
    distance=0.6,
    return_metadata=wvc.query.MetadataQuery(distance=True),
    limit=2
)

for o in response.objects:
    print(o.properties)
    print(o.metadata)

QueryReturn(objects=[Object(uuid=_WeaviateUUIDInt('7155cb5f-9f3c-441f-a99d-1efa45e1617b'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=0.5686601400375366, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'text': 'bar graph, mobile accesories orders analysis, pouches, holsters, shells, skins, fitted cases, bumpers, flip cases, sleeves', 'name': 'Mobile Accessories Orders Analysis'}, references=None, vector={}, collection='DemoCollection'), Object(uuid=_WeaviateUUIDInt('4be11d39-ac16-4ae8-9ca0-786f64d2594a'), metadata=MetadataReturn(creation_time=None, last_update_time=None, distance=0.5763142704963684, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None), properties={'text': 'yearly expenses, insurance, utilities, groceries', 'name': 'yearly expenses'}, references=None, vector={}, collection='DemoCollection')])
[Object(uuid=_WeaviateUUIDInt('7155cb5f-9f3c-441f-a99d-1efa45

In [31]:
import weaviate.classes as wvc
from weaviate.classes.query import Move

collection = client.collections.get(collection_name)

response = collection.query.near_text(
    query="pie chart",
    distance=0.6,
    # move_to=Move(force=0.85, concepts="enemy"),
    # move_away=Move(force=0.45, concepts="friends"),
    return_metadata=wvc.query.MetadataQuery(distance=True),
    limit=2
)

for o in response.objects:
    print(o.properties)
    print(o.metadata)

{'text': None, 'name': 'chart2.png'}
MetadataReturn(creation_time=None, last_update_time=None, distance=0.3431311845779419, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)
{'text': None, 'name': 'chart1.png'}
MetadataReturn(creation_time=None, last_update_time=None, distance=0.37204670906066895, certainty=None, score=None, explain_score=None, is_consistent=None, rerank_score=None)
